In [1]:
from genetic_programming import get_variable_values
from data_loader import load_acc_data, load_params, load_vol_data
from classifiers import get_default_classifiers, set_classifier, get_online_predictions
from utilities import get_all_windows, extract_windows_data, apply_volume_filter, rename_columns, classify_by_access, evaluate_model

ModuleNotFoundError: No module named 'Framework0824'

In [ ]:
params = load_params()
df = load_acc_data("Pop2")
df_vol = load_vol_data("Pop2")
print(params)
df.head()

In [ ]:
if params['model'] == "GP":
    # Configuração específica para o Genetic Programming
    print("Executing Genetic Programming model")
    
    # Preparando os dados de treino, validação e teste
    number_of_weeks = len(df.columns) - 1  # Ignorando a coluna de namespace
    last_week_of_training = number_of_weeks - params['window_size'] * 2

    best_individual = None
    best_metrics = None
    
    for first_week in range(0, last_week_of_training, params['step_size']):
        windows = get_all_windows(first_week, params['window_size'], params['step_size'])
        last_week_of_training = windows['x_train'][1]
        
        # Extrair os dados das janelas
        data_dict = extract_windows_data(df, df_vol, windows)
        
        # Aplicar o filtro de volume nos dados
        filtered_data = apply_volume_filter(data_dict, df_vol, last_week_of_training)
        
        # Renomear as colunas para serem consistentes
        renamed_data = rename_columns(filtered_data)
        
        # Transformar os dados de rótulo em binário
        renamed_data['y_train'] = classify_by_access(renamed_data['y_train'])
        renamed_data['y_test'] = classify_by_access(renamed_data['y_test'])
        
        # Executar o algoritmo genético
        training_set = get_variable_values(renamed_data['x_train'], first_week)
        generations = 100  # Número de gerações para o algoritmo genético, pode ser ajustado
        pop_size = 50  # Tamanho da população, pode ser ajustado
        best_individual = genetic_algorithm(pop_size, generations, first_week, training_set)
        
        # Obter as previsões do melhor indivíduo encontrado
        y_pred = [execute_expression(best_individual) for _ in range(len(renamed_data['x_test']))]
        y_prob = y_pred
        
        # Avaliar o modelo com várias métricas
        metrics = evaluate_model(renamed_data['y_test'].ravel(), y_pred, y_prob)
        
        if best_metrics is None or metrics['accuracy'] > best_metrics['accuracy']:
            best_metrics = metrics

elif params['model'] == "ONL":
    print("ONL")
    
    number_of_weeks = len(df.columns) - 1  # Ignorando a coluna de namespace
    last_week_of_training = number_of_weeks - params['window_size'] * 2

    for first_week in range(0, last_week_of_training, params['step_size']):
        windows = get_all_windows(first_week, params['window_size'], params['step_size'])
        last_week_of_training = windows['x_train'][1]
        
        # Extrair os dados das janelas
        data_dict = extract_windows_data(df, df_vol, windows)
        
        # Aplicar o filtro de volume nos dados
        filtered_data = apply_volume_filter(data_dict, df_vol, last_week_of_training)
        
        # Renomear as colunas para serem consistentes
        renamed_data = rename_columns(filtered_data)
        
        # Transformar os dados de rótulo em binário
        renamed_data['y_train'] = classify_by_access(renamed_data['y_train'])
        renamed_data['y_test'] = classify_by_access(renamed_data['y_test'])
        
        # Treinar o modelo
        y_pred = get_online_predictions(renamed_data['x_test'])
        y_prob = y_pred
        
        # Avaliar o modelo com várias métricas
        metrics = evaluate_model(renamed_data['y_test'].ravel(), y_pred, y_prob)
else:
    clfs_dict = get_default_classifiers()
    clf = set_classifier(params['model'], clfs_dict)

    number_of_weeks = len(df.columns) - 1  # Ignorando a coluna de namespace
    last_week_of_training = number_of_weeks - params['window_size'] * 2

    for first_week in range(0, last_week_of_training, params['step_size']):
        windows = get_all_windows(first_week, params['window_size'], params['step_size'])
        last_week_of_training = windows['x_train'][1]
        
        # Extrair os dados das janelas
        data_dict = extract_windows_data(df, df_vol, windows)
        
        # Aplicar o filtro de volume nos dados
        filtered_data = apply_volume_filter(data_dict, df_vol, last_week_of_training)
        
        # Renomear as colunas para serem consistentes
        renamed_data = rename_columns(filtered_data)
        
        # Transformar os dados de rótulo em binário
        renamed_data['y_train'] = classify_by_access(renamed_data['y_train'])
        renamed_data['y_test'] = classify_by_access(renamed_data['y_test'])
        
        # Treinar o modelo
        clf.fit(renamed_data['x_train'], renamed_data['y_train'])
        y_prob = clf.predict_proba(renamed_data['x_test'])[:, 1]  # Pegando a probabilidade da classe positiva
        
        # Testar o modelo
        y_pred = clf.predict(renamed_data['x_test'])
        
        # Avaliar o modelo com várias métricas
        metrics = evaluate_model(renamed_data['y_test'].ravel(), y_pred, y_prob)

# Exibir as métricas finais
print(metrics)
